In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install pmdarima

# 1. **모듈 불러오기**

In [ ]:
import os
import pandas as pd
import pandas_datareader as pdr

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('seaborn-whitegrid')

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima

import seaborn as sns

plt.style.use('seaborn-whitegrid')
sns.set_style('white')
%matplotlib inline
import itertools
import warnings
warnings.filterwarnings(action='ignore')


# **2. 데이터 로딩**

In [ ]:
data = pd.read_csv('../input/air-passengers/AirPassengers.csv')
data.head()


In [ ]:
# 데이터 로딩하고 month 를 datetime 객체로 바꾸고 index로 지정한다.
data = data.rename(columns = {'Month':'month', '#Passengers':'passengers'})
data['month'] = pd.to_datetime(data['month'])
data = data.set_index('month')
data

# **3. Box-Jenkins ARIMA Procedure**

- 3.1 Data Preprocessing
- 3.2 Identify Model to be Tentatively Entertained
- 3.3 Estimate Parameters
- 3.4 Diagnosis Check
- 3.5 use Model to Forecast

## **3.1 Data Preprocessing**

In [ ]:
type(data)

In [ ]:
data.index

In [ ]:
fig = data.plot()

In [ ]:
plt.plot(data)

In [ ]:
# 시계열 분해(계절성 분해)
# 덧셈 방식을 채택했으며 고전적 분해인것 같다.
decomposition = sm.tsa.seasonal_decompose(data['passengers'], model = 'additive',
                                          period = 1)


In [ ]:
fig = decomposition.plot()
fig.set_size_inches(10,10)
plt.show()

## **3.2 Identify Model to be Tentatively Entertained**

In [ ]:
# split
train_data, test_data = train_test_split(data, test_size = 0.2, shuffle=False)

In [ ]:
print(train_data)
print(test_data)


In [ ]:
# values 메서드를 이용하면 value 값들을 array 형태로 추출가능.
# shape 해보니까 2차원인데 한 축의 크기가 1이라 squeeze 할 것.
train_data.values.shape

In [ ]:
# ACF, PACF

fig, ax = plt.subplots(1,2,figsize = (10,5))
# 전체 fig의 title 설정
fig.suptitle('Raw data')

# 인덱스는 굳이 필요없는 듯. 어차피 시간순으로 정렬되어있음을 가정한 듯.
sm.graphics.tsa.plot_acf(train_data.values.squeeze(),lags = 30,ax=ax[0])
sm.graphics.tsa.plot_pacf(train_data.values.squeeze(), lags = 30, ax = ax[1])


딱 봐도 trend와 계절성(물결무늬)가 있어 **non-stationary**한 데이터이다. 

In [ ]:
# 차분

diff_train_data = train_data.copy()
# default 는 lag=1, na 는 제거
diff_train_data = diff_train_data.diff().dropna()
print('Raw data')
print(train_data)
print('차분한 데이터')
print(diff_train_data)

In [ ]:
# 차분 데이터 plot

plt.figure(figsize = (12,8))
# 오 이런식으로도 가능하구나. 
# subplot 바로 적용하고 세로2가로1에서 첫번째로 지정한 것
plt.subplot(211)
plt.plot(train_data['passengers'])
plt.legend(['Raw data(Non-stationary)'])
plt.subplot(212)
plt.plot(diff_train_data, color= 'orange')
plt.legend(['differenced(stationary)'])

In [ ]:
# 차분한 데이터의 ACF, PACF

fig, ax = plt.subplots(1,2,figsize=(12,8))
fig.suptitle('differenced data')

sm.graphics.tsa.plot_acf(diff_train_data, ax = ax[0])
sm.graphics.tsa.plot_pacf(diff_train_data, ax = ax[1])

**stationary** 하구만
잘 보면 ACF 가 <u>소멸하는 sin 함수</u> 처럼 생겼다.   
이 사실로 우리는 대략적인 모델을 설정할 수 있다.

|Model|ACF|PACF|
|:---|:---:|---:|
|MA(q)|cut off after lag q|Die out|
|AR(p)|Die out|cut off after lag p|
|ARMA(p,q)|Die out  (after q-p)|Die out  (after q-p)|


`Die out` : exponentially or sin function

ARMA 가 적절해보이는데 강의자는 AR(p) 로 일단 모델 설정  
코드상으로는 `ARIMA(1,1,0)` 이 될 것.

In [ ]:
# 신가하게 모델에 바로 train_data 를 집어넣고 fit()을 취해준다.
model = ARIMA(train_data, order = (1,1,0))
model_fit = model.fit()
model_fit.summary()

여기서 ARIMA 모델의 계수 및 상수값을 알 수 있고, 그에 대한 p-value도 알 수 있다. 원래 여기서는 constant가 유효하지 않으므로,<br>
`model.fit(trend = 'nc')` 로 하는 것이 옳다.

## 3.4.1 Diagnosis Check - ARIMA

In [ ]:
# parameter search

p = range(0,3)
d = range(1,2)
q = range(0,3)

# itertools 는 이터레이션을 도와주는 라이브러리로,
# itertools.product 는 인수로 들어간 것들의 모든 조합을 반환한다.
pdq = list(itertools.product(p,d,q))
print(pdq)

aic = []
for i in pdq:
    model = ARIMA(train_data, order = i)
    model_fit = model.fit()
    print('ARIMA: {0} >> AIC : {1}'.format(i, round(model_fit.aic,2)))
    aic.append(round(model_fit.aic,2))

In [ ]:
optimal = [(pdq[i], j) for i, j in enumerate(aic) if j == min(aic)]
optimal[0]

In [ ]:
model_opt = ARIMA(train_data, order = optimal[0][0])
model_opt_fit = model_opt.fit()
model_opt_fit.summary()

요번에는 p-value 가 아주 만족스럽게 나옴.

## 3.5.1 use Model to Forcast - ARIMA

`.forecast`는 3개의 array를 반환하며
1. 첫번째 array
  - 점예측값을 반환
2. 두번째 array
  - 표준오차(standard error)를 반환
3. 세번째 array
  - 두번째에서 구한 표준오차를 통해 prediction interval 범위를 반환. [lower bound, upper bound]의 형태

In [ ]:
pred = model_opt_fit.forecast(len(test_data))
print(pred)

In [ ]:
# forecast라는 메서드를 많이 쓴다고 하며, 들어가는 인수는 이 후로 얼마나 예측할지이다.
# predict 메서드도 존재하며 이는 start와 end index를 집어넣어 값을 도출한다.

# .forecast 는 3개의 array를 튜플로 묶은 데이터를 반환한다. 
# 첫번째 array 는 점예측값이고 두번째는 upper bound, 세번째는 lower bound 이다. 
pred = model_opt_fit.forecast(len(test_data))
pred_value = pred[0]

pred_ub = pred[2][:,0]
pred_lb = pred[2][:,1]
pred_index = list(test_data.index)
r2 = r2_score(test_data, pred_value)

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
ax.plot(data.index, data.passengers)
# 그냥 날짜 넣으니까 인식몬해서 이렇게 함.
ax.vlines(pd.to_datetime('1958-08-01'), 0, 1000,linestyle = '--', color='r', 
         label='Start of Forecast');
ax.plot(pred_index, pred_value, label = 'prediction',color='orange')
# 요걸로 범위 채우기
ax.fill_between(pred_index, pred_lb, pred_ub, color='k', alpha=0.1,
               label='0.95 prediciton interval')
ax.legend(loc='upper left')
plt.suptitle(f'ARIMA {optimal[0][0]} Prediction Results (r2_score:{round(r2,2)})')
plt.show()

강의 그대로하니까 뭔가 안됐는데, 저런식으로 하니까 되네

## 3.4.2 Diagnosis Check - SARIMA

지금까지는 계절성을 반영하지 않았다. 그러나 SARIMA 에서는 계절성을 반영할 수 있다.

In [ ]:
# 우선 어떤 parameter 를 넣을지 search 해보자

print('Examples of parameter combinations of SARIMA...')
p = range(0,3)
d = range(1,2)
q = range(0,3)
s = range(12,13)

pdq = list(itertools.product(p,d,q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]
print(pdq)

aic = []
params = []
for i in pdq:
    for j in seasonal_pdq:
        try:
            model = SARIMAX(train_data.values, order=(i), seasonal_order=(j))
            model_fit = model.fit()
            print(f'SARIMA parameter : {i} {j} >> AIC: {round(model_fit.aic,2)}')
            params.append((i,j))
            aic.append(round(model_fit.aic,2))
        except:
            continue
            

In [ ]:
optimal = [(params[i], j) for i, j in enumerate(aic) if j==min(aic)]
optimal

In [ ]:
model_opt = SARIMAX(train_data, order = optimal[0][0][0],
                   seasonal_order = optimal[0][0][1])
model_opt_fit = model_opt.fit()
model_opt_fit.summary()

## 3.5.2 use Model to Forecast - SARIMA

In [ ]:
model_opt_fit.get_forecast(len(test_data))

In [ ]:
# ARIMA와 코드가 살짝 다름(forcast 대신 get_forcast)
prediction = model_opt_fit.get_forecast(len(test_data))
predicted_value = prediction.predicted_mean
predicted_ub = prediction.conf_int().iloc[:,0]
predicted_lb = prediction.conf_int().iloc[:,1]
predict_index = list(test_data.index)
r2 = r2_score(test_data, predicted_value)

In [ ]:
fig, ax= plt.subplots(figsize=(12,6))
ax.plot(data.index, data)
ax.vlines(pd.to_datetime('1958-08-01'), 0,700, linestyle ='--', color='r',
         label = 'Start of  Forcast')
ax.plot(predict_index, predicted_value, label='prediction')
ax.fill_between(predict_index, predicted_lb, predicted_ub, color='k',
               alpha= 0.1, label='0.95 prediction interval')
ax.legend(loc='upper left')
plt.suptitle(f'SARIMA {optimal[0][0][0], optimal[0][0][1]} prediction results (r2score: {round(r2,2)})')
plt.show()
             

계절성을 반영하니 더 좋은 모델이 만들어졌네~

## 3.4.3 Diagnosis Check - auto arima

```python
print('Examples of parameter combinations of SARIMA...')
p = range(0,3)
d = range(1,2)
q = range(0,3)
s = range(12,13)

pdq = list(itertools.product(p,d,q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]
print(pdq)

aic = []
params = []
for i in pdq:
    for j in seasonal_pdq:
        try:
            model = SARIMAX(train_data.values, order=(i), seasonal_order=(j))
            model_fit = model.fit()
            print(f'SARIMA parameter : {i} {j} >> AIC: {round(model_fit.aic,2)}')
            params.append((i,j))
            aic.append(round(model_fit.aic,2))
        except:
            continue
```

 위 코드가 비효율적이라고 하면서 새로운 패키지를 소개해줌(auto arima)

In [ ]:
# Parameter search
# trace 는 결과값을 print 해주는 인자
# 이전에는 AIC가 가장 작은 경우의 파라미터를 다시 넣어서 피팅했는데
# auto_arima 는 자동으로 가장 적합한 파라미터를 학습시킴

auto_arima_model = auto_arima(train_data, start_p=1, start_q=1, 
                             max_p = 3, max_q = 3, m=12, seasonal=True,
                             d=1, D=1, max_P=3, max_Q = 3,
                             trace=True,
                             error_action = 'ignore',
                             suppress_warnings = True,
                             stepwise=False)

In [ ]:
auto_arima_model.summary()

## 3.5.3 use Model to Forecast - auto_arima

In [ ]:
# predict를 쓴다.
prediction = auto_arima_model.predict(len(test_data), return_conf_int=True)
print(prediction)

In [ ]:
predicted_value = prediction[0]
predicted_ub = prediction[1][:,0]
predicted_lb = prediction[1][:,1]
predicted_index = list(test_data.index)
r2 = r2_score(test_data, predicted_value)

In [ ]:
fig, ax = plt.subplots(figsize= (12,6))
ax.plot(data.index, data)
ax.vlines(pd.to_datetime('1958-08-01'), 0, 700, linestyle='--',
         color='r', label='Start of Forecast')
ax.plot(predicted_index, predicted_value, label='Prediction')
ax.fill_between(predicted_index, predicted_ub, predicted_lb,
               color='k', alpha=1, label='interval')
ax.legend(loc='upper left')
plt.suptitle(f'SARIMA({auto_arima_model.order, auto_arima_model.seasonal_order}) prediction results: {round(r2,2)}(r2 score)' )

# 4. Case Study: 주가예측

- 데이터: 5년간 주가 시계열 데이터

In [ ]:
# 일단 기업명을 넣었을 때, 종목코드를 반환하는 함수를 만들자
# query 에 조건을 넣으면 

def get_code(df, corp):
    code=df.query("name=='{}'".format(corp))['code'].to_string(index=False)
    code=code.strip()
    return code

In [ ]:
raw_data = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
raw_data

In [ ]:
code_data1 = raw_data[['회사명','종목코드']]
code_data = code_data1.rename(columns={'회사명':'name', '종목코드':'code'})
# 종목코드를 6자리로 통일
code_data.code = code_data.code.map('{:06d}'.format)

In [ ]:
code_data

In [ ]:
code_data[code_data.name == 'LG이노텍']

In [ ]:
stock_code = get_code(code_data,'LG이노텍')
stock_code = stock_code + '.KS'
stock_data = pdr.get_data_yahoo(stock_code)
stock_data

In [ ]:
stock_data = stock_data[['Close']]
# 이런 식으로도 plot그리기 가능
fig = stock_data.plot()

In [ ]:
stock_data.values

In [ ]:
# Y label이 없으므로 2개로만 나뉨
# shuffle = False 해줘야 시계열의 뒷부분만 따로 test set으로 빠짐
from sklearn.model_selection import train_test_split
stock_data_train, stock_data_test = train_test_split(stock_data,
                                                    test_size=0.2,
                                                    shuffle=False)


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6))
fig.suptitle('Raw data')
# 인자로 ndarray 를 넣어야해서 그런가봄
sm.graphics.tsa.plot_acf(stock_data_train.values.squeeze(), lags=40,
                        ax= ax[0])
sm.graphics.tsa.plot_pacf(stock_data_train.values.squeeze(), lags=40,
                         ax= ax[1])


ACF가 점차적으로 줄어드는 것으로 보아 **non-stationary** 이다.

In [ ]:
diff_stock_data_train = stock_data_train.copy()
diff_stock_data_train = diff_stock_data_train.diff()
diff_stock_data_train = diff_stock_data_train.dropna()
print('####Raw data####')
print(stock_data_train)
print('####diffed data####')
print(diff_stock_data_train)

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(12,8))
ax[0].plot(stock_data_train)
ax[0].legend(['Raw Data(Non-stationary)'])
ax[1].plot(diff_stock_data_train, color='r')
ax[1].legend(['Diffed Data(Stationaru)'])
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,6))
fig.suptitle('Diffed')
sm.graphics.tsa.plot_acf(diff_stock_data_train.values.squeeze(),
                        lags =  40, ax=ax[0])
sm.graphics.tsa.plot_pacf(diff_stock_data_train.values.squeeze(),
                         lags=40, ax=ax[1])


In [ ]:
auto_arima_model = auto_arima(stock_data_train,start_p=1,
                             start_q = 1,
                             seasonal=False, d = 1,trace=True,
                             error_action = 'ignore',
                             suppress_warnings = True,
                             stepwise=False)

In [ ]:
auto_arima_model.summary()

In [ ]:
# return_conf_int 는 confidential interval을 출력한다는 의미인듯
pred = auto_arima_model.predict(len(stock_data_test),
                               return_conf_int = True)
print(pred)

In [ ]:
pred_value = pred[0]
pred_lb = pred[1][:,0]
pred_ub = pred[1][:,1]
pred_index = list(stock_data_test.index)
pred_index[0]


In [ ]:
fig, ax = plt.subplots(figsize = (12,6))
ax.plot(stock_data.index, stock_data)
ax.vlines(pd.to_datetime('2020-01-31'), 50000, 225000,
         linestyle = '--', color = 'r', label = 'Start of Forecast')
ax.plot(pred_index, pred_value, label = 'prediction')
ax.fill_between(pred_index, pred_lb, pred_ub, color='k', alpha = 0.1,
               label='0.95 pred interval')
plt.suptitle(f'ARIMA {auto_arima_model.order}, prediction results')
plt.show()


# EOD